In [7]:
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.applications.densenet import DenseNet121
from nasnet_keras import NASNet
from keras.optimizers import SGD, Adam
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf
import os
from keras import backend as K
import numpy as np
from cutout import cutout

os.environ["CUDA_VISIBLE_DEVICES"] = '2'
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)

In [8]:
BATCH_SIZE = 128
NUM_CLASSES = 10
epochs = 200
data_augmentation = True
img_rows, img_cols, img_channels = (32, 32, 3)

(x_train, y_train), (x_test, y_test) = cifar10.load_data()


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = to_categorical(y_train, NUM_CLASSES)
y_test = to_categorical(y_test, NUM_CLASSES)

nasnet = NASNet(input_shape=(32, 32, 3), classes=10)
optimizer = SGD(lr=1e-3, momentum=0.9)
nasnet.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
nasnet.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_22 (Conv2D)              (None, 32, 32, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 32, 32, 32)   128         conv2d_22[0][0]                  
__________________________________________________________________________________________________
activation_238 (Activation)     (None, 32, 32, 32)   0           stem_bn1[0][0]                   
__________________________________________________________________________________________________
conv2d_23 

reduction_left1_6 (MaxPooling2D (None, 16, 16, 64)   0           batch_normalization_3[0][0]      
__________________________________________________________________________________________________
separable_conv_2_bn_reduction_r (None, 16, 16, 64)   256         separable_conv_2_reduction_right1
__________________________________________________________________________________________________
reduction_left3_6 (AveragePooli (None, 16, 16, 64)   0           add_133[0][0]                    
__________________________________________________________________________________________________
add_132 (Add)                   (None, 16, 16, 64)   0           reduction_left4_6[0][0]          
                                                                 separable_conv_2_bn_reduction_rig
__________________________________________________________________________________________________
reduction_left5_6 (AveragePooli (None, 16, 16, 64)   0           batch_normalization_3[0][0]      
__________

In [9]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.5), cooldown=0, patience=5, min_lr=0.5e-5)

In [10]:
datagen = ImageDataGenerator(featurewise_center=False, 
                             samplewise_center=False,
                             featurewise_std_normalization=False, 
                             samplewise_std_normalization=False,
                             zca_whitening=False, 
                             rotation_range=0, 
                             width_shift_range=0.1, 
                             height_shift_range=0.1,
                             horizontal_flip=False, 
                             vertical_flip=False,
                             preprocessing_function=cutout)

In [ ]:
datagen.fit(x_train)

In [ ]:
nasnet.fit_generator(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE), steps_per_epoch=x_train.shape[0] // BATCH_SIZE,
                    validation_data=(x_test, y_test), epochs=epochs, callbacks=[lr_reducer])

Epoch 1/200
390/390 [==============================] - 351s 901ms/step - loss: 20.8193 - acc: 0.1839 - val_loss: 23.9069 - val_acc: 0.1164
Epoch 2/200
390/390 [==============================] - 233s 597ms/step - loss: 20.3735 - acc: 0.2695 - val_loss: 23.3339 - val_acc: 0.1329
Epoch 3/200
390/390 [==============================] - 231s 593ms/step - loss: 20.1109 - acc: 0.3110 - val_loss: 21.9092 - val_acc: 0.1344
Epoch 4/200
390/390 [==============================] - 223s 572ms/step - loss: 19.8879 - acc: 0.3498 - val_loss: 22.7567 - val_acc: 0.1201
Epoch 5/200
390/390 [==============================] - 220s 564ms/step - loss: 19.6947 - acc: 0.3709 - val_loss: 20.6197 - val_acc: 0.1847
Epoch 6/200
390/390 [==============================] - 220s 564ms/step - loss: 19.5078 - acc: 0.3903 - val_loss: 19.9871 - val_acc: 0.2552
Epoch 7/200
390/390 [==============================] - 220s 563ms/step - loss: 19.3381 - acc: 0.4038 - val_loss: 20.0707 - val_acc: 0.2309
Epoch 8/200
390/390 [======

390/390 [==============================] - 218s 559ms/step - loss: 12.8601 - acc: 0.6307 - val_loss: 13.6698 - val_acc: 0.4445
Epoch 60/200
390/390 [==============================] - 217s 558ms/step - loss: 12.7516 - acc: 0.6375 - val_loss: 13.5759 - val_acc: 0.4407
Epoch 61/200
390/390 [==============================] - 218s 558ms/step - loss: 12.6615 - acc: 0.6383 - val_loss: 13.8795 - val_acc: 0.4127
Epoch 62/200
390/390 [==============================] - 218s 558ms/step - loss: 12.5735 - acc: 0.6346 - val_loss: 13.4335 - val_acc: 0.3972
Epoch 63/200
390/390 [==============================] - 218s 559ms/step - loss: 12.4874 - acc: 0.6354 - val_loss: 13.1816 - val_acc: 0.4207
Epoch 64/200
390/390 [==============================] - 218s 559ms/step - loss: 12.3737 - acc: 0.6424 - val_loss: 13.0421 - val_acc: 0.4736
Epoch 65/200
390/390 [==============================] - 218s 559ms/step - loss: 12.2926 - acc: 0.6420 - val_loss: 14.7578 - val_acc: 0.3033
Epoch 66/200
390/390 [===========

In [ ]:
nasnet.save("nasnet(6@767)_cifar10_cutout")